# Query EFD ATPneumatics pressures
Craig Lage - 09-Sep-24

In [ ]:
import sys, time, os, asyncio, glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient

In [ ]:
client = EfdClient('usdf_efd')

In [ ]:
start = Time("2022-01-01 00:00:00Z", scale='utc')
end = Time("2024-09-01 00:00:00Z", scale='utc')
main = await client.select_time_series('lsst.sal.ATPneumatics.mainAirSourcePressure', ['pressure'], start, end)
len(main)

In [ ]:
pascal_to_psi = 0.000145038
plt.title("AuxTel Main Air Source Pressure")
main_psi = main['pressure'] * pascal_to_psi
main_psi.plot()
plt.ylabel("Main air pressure (PSI)")
plt.savefig("/home/c/cslage/u/AuxTel/atpneumatics/Main_Air_Pressure_2022-2024.png")

In [ ]:
start = Time("2024-08-01 00:00:00Z", scale='utc')
end = Time("2024-09-01 00:00:00Z", scale='utc')
m1 = await client.select_time_series('lsst.sal.ATPneumatics.m1AirPressure', ['pressure'], start, end)
m2 = await client.select_time_series('lsst.sal.ATPneumatics.m2AirPressure', ['pressure'], start, end)
print(len(m1), len(m2))

In [ ]:
plt.subplots_adjust(hspace=0.8)
plt.subplot(2,1,1)
plt.title("AuxTel M1 Pressure")
m1_psi = m1['pressure'] * pascal_to_psi
m1_psi.plot()
plt.ylabel("M1 pressure (PSI)")
plt.subplot(2,1,2)
plt.title("AuxTel M2 Pressure")
m2_psi = m2['pressure'] * pascal_to_psi
m2_psi.plot()
plt.ylabel("M2 pressure (PSI)")
plt.savefig("/home/c/cslage/u/AuxTel/atpneumatics/M1_M2_Pressure_Aug2024.png")

In [ ]:
def m1_pressure(el):
    pascal_to_psi = 0.000145038
    m1_coefs = [114351.586, -220485.725, 278187.272, -48810.102]
    pressure = m1_coefs[0] + m1_coefs[1] * np.cos((90 - el) * np.pi / 180.0) + \
                  m1_coefs[2] * np.cos((90 - el) * np.pi / 180.0)**2 + \
                    m1_coefs[3] * np.cos((90 - el) * np.pi / 180.0)**3
    return pressure * pascal_to_psi

In [ ]:
start = Time("2024-08-21 20:30:00Z", scale='utc')
end = Time("2024-08-22 05:00:00Z", scale='utc')
m1 = await client.select_time_series('lsst.sal.ATPneumatics.m1AirPressure', ['pressure'], start, end)
m2 = await client.select_time_series('lsst.sal.ATPneumatics.m2AirPressure', ['pressure'], start, end)
el = await client.select_time_series("lsst.sal.ATMCS.logevent_target", \
                                            ["elevation"],  start, end)
print(len(m1), len(m2), len(el))

In [ ]:
pascal_to_psi = 0.000145038
plt.subplots_adjust(hspace=0.8)
plt.subplot(2,1,1)
plt.title("AuxTel M1 Pressure")
m1_psi = m1['pressure'] * pascal_to_psi
m1_psi.plot(label='M1 pressure')
plt.ylabel("M1 pressure (PSI)")
times = []
LUTs = []
for i in range(0, len(el), 10000):
    elValue = el['elevation'].values[i]
    LUT_pressure = m1_pressure(elValue)
    LUTs.append(LUT_pressure)
    times.append(el.index[i])
plt.scatter(times, LUTs, label='LUT', color='black', marker='x')
plt.legend()    
plt.subplot(2,1,2)
plt.title("AuxTel M2 Pressure")
m2_psi = m2['pressure'].rolling(100).sum() * pascal_to_psi
m2_psi.plot()
plt.ylabel("M2 pressure (PSI)")
plt.legend()
plt.savefig("/home/c/cslage/u/AuxTel/atpneumatics/M1_M2_Pressure_21Aug2024.png")

In [ ]:
plot_el = np.linspace(0, 86, 100)
plt.title("AuxTel M1 LUT pressure")
plt.plot(plot_el, m1_pressure(plot_el))
plt.xlabel("Elevation (degrees)")
plt.ylabel("LUT pressure (PSI)")
plt.savefig("/home/c/cslage/u/AuxTel/atpneumatics/M1_LUT_Pressure_.png")